<h1>Getting Started With SQL and BigQuery</h1>

<h2>Introduction</h2>

Structured Query Language, or SQL, is the programming language used with databases, and it is an important skill for any data scientist. In this course, you'll build your SQL skills using BigQuery, a web service that lets you apply SQL to huge datasets.

In this lesson, you'll learn the basics of accessing and examining BigQuery datasets. After you have a handle on these basics, we'll come back to build your SQL skills.



<h2>Your first BigQuery commands</h2>

To use BigQuery, we'll import the Python package below:

In [ ]:
from google.cloud import bigquery

The first step in the workflow is to create a `Client object`. As you'll soon see, this `Client object` will play a central role in retrieving information from BigQuery datasets.

In [ ]:
# Create a "Client" object
client = bigquery.Client()

Authenticate

In [ ]:
from google.colab import auth
auth.authenticate_user()

We'll work with a dataset of posts on [Hacker News](https://news.ycombinator.com/), a website focusing on computer science and cybersecurity news.

In BigQuery, each dataset is contained in a corresponding project. In this case, our `hacker_news` dataset is contained in the `bigquery-public-data` project.

We use the `list_datasets` method to list all the datasets in the project.

In [ ]:
datasets = list(client.list_datasets(project="bigquery-public-data"))  # Make an API request.

In [ ]:
project = client.project

if datasets:
    print("Datasets in project {}:".format(project))
    for dataset in datasets:
        print("\t{}".format(dataset.dataset_id))
else:
    print("{} project does not contain any datasets.".format(project))

Datasets in project :
	america_health_rankings
	austin_311
	austin_bikeshare
	austin_crime
	austin_incidents
	austin_waste
	baseball
	bbc_news
	bigqueryml_ncaa
	bitcoin_blockchain
	blackhole_database
	blockchain_analytics_ethereum_mainnet_us
	bls
	bls_qcew
	breathe
	broadstreet_adi
	catalonian_mobile_coverage
	catalonian_mobile_coverage_eu
	census_bureau_acs
	census_bureau_construction
	census_bureau_international
	census_bureau_usa
	census_opportunity_atlas
	census_utility
	cfpb_complaints
	chicago_crime
	chicago_taxi_trips
	clemson_dice
	cloud_storage_geo_index
	cms_codes
	cms_medicare
	cms_synthetic_patient_data_omop
	country_codes
	covid19_aha
	covid19_covidtracking
	covid19_ecdc
	covid19_ecdc_eu
	covid19_genome_sequence
	covid19_geotab_mobility_impact
	covid19_geotab_mobility_impact_eu
	covid19_google_mobility
	covid19_google_mobility_eu
	covid19_govt_response
	covid19_italy
	covid19_italy_eu
	covid19_jhu_csse
	covid19_jhu_csse_eu
	covid19_nyt
	covid19_open_data
	covid19_open_data

To access the dataset,

*   We begin by constructing a reference to the dataset with the `dataset()` method.
*   Next, we use the `get_dataset()` method, along with the reference we just constructed, to fetch the dataset.


In [ ]:
# Construct a reference to the "hacker_news" dataset
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

Every dataset is just a collection of tables. You can think of a dataset as a spreadsheet file containing multiple tables, all composed of rows and columns.

We use the `list_tables()` method to list the tables in the dataset.

In [ ]:
# List all the tables in the "hacker_news" dataset
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset (there are four!)
for table in tables:
    print(table.table_id)

full


Similar to how we fetched a dataset, we can fetch a table. In the code cell below, we fetch the `full` table in the `hacker_news` dataset.

In [ ]:
# Construct a reference to the "full" table
table_ref = dataset_ref.table("full")

# API request - fetch the table
table = client.get_table(table_ref)


<h2>Table schema</h2>

The structure of a table is called its **schema**. We need to understand a table's schema to effectively pull out the data we want.

In this example, we'll investigate the `full` table that we fetched above.

In [ ]:
# Print information on all the columns in the "full" table in the "hacker_news" dataset
table.schema

[SchemaField('title', 'STRING', 'NULLABLE', None, 'Story title', (), None),
 SchemaField('url', 'STRING', 'NULLABLE', None, 'Story url', (), None),
 SchemaField('text', 'STRING', 'NULLABLE', None, 'Story or comment text', (), None),
 SchemaField('dead', 'BOOLEAN', 'NULLABLE', None, 'Is dead?', (), None),
 SchemaField('by', 'STRING', 'NULLABLE', None, "The username of the item's author.", (), None),
 SchemaField('score', 'INTEGER', 'NULLABLE', None, 'Story score', (), None),
 SchemaField('time', 'INTEGER', 'NULLABLE', None, 'Unix time', (), None),
 SchemaField('timestamp', 'TIMESTAMP', 'NULLABLE', None, 'Timestamp for the unix time', (), None),
 SchemaField('type', 'STRING', 'NULLABLE', None, 'type of details (comment comment_ranking poll story job pollopt)', (), None),
 SchemaField('id', 'INTEGER', 'NULLABLE', None, "The item's unique id.", (), None),
 SchemaField('parent', 'INTEGER', 'NULLABLE', None, 'Parent comment ID', (), None),
 SchemaField('descendants', 'INTEGER', 'NULLABLE', N

Each `SchemaField` tells us about a specific column (which we also refer to as a **field**). In order, the information is:

* The **name** of the column
* The **field type** (or datatype) in the column
* The **mode** of the column (`'NULLABLE'` means that a column allows NULL values, and is the default)
* A **description** of the data in that column

The first field has the SchemaField:

`SchemaField('by', 'string', 'NULLABLE', "The username of the item's author.",())`

This tells us:

* the field (or column) is called `by`,
* the data in this field is strings,
* NULL values are allowed, and
* it contains the usernames corresponding to each item's author.

We can use the [list_rows()](https://google-cloud.readthedocs.io/en/latest/bigquery/generated/google.cloud.bigquery.client.Client.list_rows.html#google.cloud.bigquery.client.Client.list_rows) method to check just the first five lines of of the full table to make sure this is right. (Sometimes databases have outdated descriptions, so it's good to check.) This returns a BigQuery [RowIterator](https://googleapis.github.io/google-cloud-python/latest/bigquery/generated/google.cloud.bigquery.table.RowIterator.html) object that can quickly be converted to a pandas DataFrame with the [to_dataframe](https://google-cloud.readthedocs.io/en/latest/bigquery/generated/google.cloud.bigquery.table.RowIterator.to_dataframe.html) method.

In [ ]:
# Preview the first five lines of the "full" table
client.list_rows(table, max_results=5).to_dataframe()

,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,None,None,None,<NA>,None,<NA>,1260720268,2009-12-13 16:04:28+00:00,story,992806,<NA>,<NA>,<NA>,<NA>
1,None,None,None,<NA>,None,<NA>,1437549870,2015-07-22 07:24:30+00:00,story,9928163,<NA>,<NA>,<NA>,<NA>
2,None,None,None,<NA>,None,<NA>,<NA>,NaT,story,99282,<NA>,<NA>,<NA>,<NA>
3,None,None,None,<NA>,None,<NA>,1437551804,2015-07-22 07:56:44+00:00,story,9928251,<NA>,<NA>,<NA>,<NA>
4,None,None,None,<NA>,None,<NA>,1437551962,2015-07-22 07:59:22+00:00,story,9928258,<NA>,<NA>,<NA>,<NA>
